Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [38]:
## LR with L2
batch_size = 128
num_steps = 3001
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  x = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  b = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(x, w) + b 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits) + beta * tf.nn.l2_loss(w))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid, w) + b)
  test_prediction = tf.nn.softmax(tf.matmul(test, w) + b)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 51.111572
Minibatch accuracy: 7.0%
Validation accuracy: 8.4%
Minibatch loss at step 500: 11.384155
Minibatch accuracy: 78.9%
Validation accuracy: 72.4%
Minibatch loss at step 1000: 4.524247
Minibatch accuracy: 78.1%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 1.832128
Minibatch accuracy: 82.8%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 1.052330
Minibatch accuracy: 88.3%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.884340
Minibatch accuracy: 79.7%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.799428
Minibatch accuracy: 83.6%
Validation accuracy: 82.2%
Test accuracy: 89.1%


In [39]:
## NN with L2
batch_size = 128
layer1_dimension = 1024
num_steps = 3001
beta = 0.01

graph = tf.Graph()
with graph.as_default():
  # 定义输入输出及参数
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_dimension]))
  b1 = tf.Variable(tf.zeros([layer1_dimension]))
  w2 = tf.Variable(tf.truncated_normal([layer1_dimension, num_labels]))  
  b2 = tf.Variable(tf.zeros([num_labels]))
  #定义训练和预测的计算过程 
  l1 = tf.nn.relu(tf.matmul(x, w1) + b1 )
  l2 = tf.matmul(l1, w2) + b2 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=l2) + beta * tf.nn.l2_loss(w1) + beta * tf.nn.l2_loss(w2))
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  train_prediction = tf.nn.softmax(l2)
  valid_relu= tf.nn.relu(tf.matmul(valid, w1) + b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, w2) + b2)
  test_relu= tf.nn.relu(tf.matmul(test, w1) + b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, w2) + b2)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 3481.892090
Minibatch accuracy: 7.0%
Validation accuracy: 22.0%
Minibatch loss at step 500: 1155.242432
Minibatch accuracy: 85.9%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 423.356720
Minibatch accuracy: 78.9%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 154.738678
Minibatch accuracy: 92.2%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 57.134701
Minibatch accuracy: 91.4%
Validation accuracy: 85.8%
Minibatch loss at step 2500: 21.445107
Minibatch accuracy: 89.1%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 8.364651
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Test accuracy: 91.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [56]:
## NN with L2, small dataset
train_size = 1024
batch_size = 128
layer1_dimension = 1024
num_steps = 3001
beta = 0.0

graph = tf.Graph()
with graph.as_default():
  # 定义输入输出及参数
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_dimension]))
  b1 = tf.Variable(tf.zeros([layer1_dimension]))
  w2 = tf.Variable(tf.truncated_normal([layer1_dimension, num_labels]))  
  b2 = tf.Variable(tf.zeros([num_labels]))
  #定义训练和预测的计算过程 
  l1 = tf.nn.relu(tf.matmul(x, w1) + b1 )
  l2 = tf.matmul(l1, w2) + b2 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=l2) + beta * tf.nn.l2_loss(w1) + beta * tf.nn.l2_loss(w2))
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  train_prediction = tf.nn.softmax(l2)
  valid_relu= tf.nn.relu(tf.matmul(valid, w1) + b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, w2) + b2)
  test_relu= tf.nn.relu(tf.matmul(test, w1) + b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, w2) + b2)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_size - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 329.298645
Minibatch accuracy: 10.2%
Validation accuracy: 21.4%
Minibatch loss at step 500: 0.000001
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Test accuracy: 80.3%


In [62]:
## NN with L2, small dataset
train_size = 1024
batch_size = 128
layer1_dimension = 1024
num_steps = 3001
beta = 0.1

graph = tf.Graph()
with graph.as_default():
  # 定义输入输出及参数
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_dimension]))
  b1 = tf.Variable(tf.zeros([layer1_dimension]))
  w2 = tf.Variable(tf.truncated_normal([layer1_dimension, num_labels]))  
  b2 = tf.Variable(tf.zeros([num_labels]))
  #定义训练和预测的计算过程 
  l1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  l2 = tf.matmul(l1, w2) + b2 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=l2) + beta * tf.nn.l2_loss(w1) + beta * tf.nn.l2_loss(w2))
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  train_prediction = tf.nn.softmax(l2)
  valid_relu= tf.nn.relu(tf.matmul(valid, w1) + b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, w2) + b2)
  test_relu= tf.nn.relu(tf.matmul(test, w1) + b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, w2) + b2)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_size - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 31860.472656
Minibatch accuracy: 15.6%
Validation accuracy: 24.4%
Minibatch loss at step 500: 2.798723
Minibatch accuracy: 83.6%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 1.358191
Minibatch accuracy: 84.4%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 1.151581
Minibatch accuracy: 93.8%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 1.273059
Minibatch accuracy: 85.2%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 1.325824
Minibatch accuracy: 85.9%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 1.304145
Minibatch accuracy: 83.6%
Validation accuracy: 79.3%
Test accuracy: 86.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [60]:
## NN with Dropout，small dataset
train_size = 1024 #train_labels.shape[0]
batch_size = 128
hidden_dimension = 1024
num_steps = 3001
beta = 0.01
keep_prob = 0.5


graph = tf.Graph()
with graph.as_default():
  # 定义输入输出及参数
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_dimension]))
  b1 = tf.Variable(tf.zeros([hidden_dimension]))
  w2 = tf.Variable(tf.truncated_normal([hidden_dimension, num_labels]))  
  b2 = tf.Variable(tf.zeros([num_labels]))

  l1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  d1 = tf.nn.dropout(l1, keep_prob)
  l2 = tf.matmul(d1, w2) + b2 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=l2) + beta * tf.nn.l2_loss(w1) + beta * tf.nn.l2_loss(w2))
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  train_prediction = tf.nn.softmax(tf.matmul(l1, w2) + b2 )
  valid_relu= tf.nn.relu(tf.matmul(valid, w1) + b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, w2) + b2)
  test_relu= tf.nn.relu(tf.matmul(test, w1) + b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, w2) + b2)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_size - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 3596.614502
Minibatch accuracy: 12.5%
Validation accuracy: 23.8%
Minibatch loss at step 500: 1154.117188
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 424.260010
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 156.009857
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 57.378876
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 21.216928
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 8.026206
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Test accuracy: 88.0%


In [65]:
## NN with Dropout and L2，full dataset
train_size = train_labels.shape[0]
batch_size = 128
hidden_dimension = 1024
num_steps = 3001
beta = 0.01
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
  # 定义输入输出及参数
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_dimension]))
  b1 = tf.Variable(tf.zeros([hidden_dimension]))
  w2 = tf.Variable(tf.truncated_normal([hidden_dimension, num_labels]))  
  b2 = tf.Variable(tf.zeros([num_labels]))

  l1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  d1 = tf.nn.dropout(l1, keep_prob)
  l2 = tf.matmul(d1, w2) + b2 
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=l2) + beta * tf.nn.l2_loss(w1) + beta * tf.nn.l2_loss(w2))
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  train_prediction = tf.nn.softmax(tf.matmul(l1, w2) + b2 )
  valid_relu= tf.nn.relu(tf.matmul(valid, w1) + b1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, w2) + b2)
  test_relu= tf.nn.relu(tf.matmul(test, w1) + b1)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, w2) + b2)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_size - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 3643.734863
Minibatch accuracy: 10.2%
Validation accuracy: 23.4%
Minibatch loss at step 500: 1148.139893
Minibatch accuracy: 89.8%
Validation accuracy: 82.6%
Minibatch loss at step 1000: 419.543365
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 153.544189
Minibatch accuracy: 89.8%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 56.745979
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 21.314857
Minibatch accuracy: 85.2%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 8.355970
Minibatch accuracy: 85.2%
Validation accuracy: 84.3%
Test accuracy: 91.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [73]:
## NN with Dropout and L2，full dataset
train_size = train_labels.shape[0]
batch_size = 128
hidden_dimension = [1024,512,256]
num_steps = 1001
beta = 0.01
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
  # 定义输入输出及参数
  x = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  valid = tf.constant(valid_dataset)
  test = tf.constant(test_dataset)
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_dimension[0]]))
  b1 = tf.Variable(tf.zeros([hidden_dimension[0]]))
  w2 = tf.Variable(tf.truncated_normal([hidden_dimension[0], hidden_dimension[1]]))  
  b2 = tf.Variable(tf.zeros([hidden_dimension[1]]))
  w3 = tf.Variable(tf.truncated_normal([hidden_dimension[1], hidden_dimension[2]]))  
  b3 = tf.Variable(tf.zeros([hidden_dimension[2]]))
  w4 = tf.Variable(tf.truncated_normal([hidden_dimension[2], num_labels]))  
  b4 = tf.Variable(tf.zeros([num_labels]))
  epsilon = tf.constant(value=0.00001, shape=b4.get_shape())
    
  l1 = tf.nn.relu(tf.matmul(x, w1) + b1)
#   d1 = tf.nn.dropout(l1, keep_prob)
  l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)
#   d2 = tf.nn.dropout(l2, keep_prob)
  l3 = tf.nn.relu(tf.matmul(l2, w3) + b3)            
#   d3 = tf.nn.dropout(l3, keep_prob)
  logit = tf.matmul(l3, w4) + b4 + epsilon
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logit))
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

  train_relu2= tf.nn.relu(tf.matmul(l1, w2) + b2)
  train_relu3= tf.nn.relu(tf.matmul(train_relu2, w3) + b3)
  train_logit= tf.matmul(train_relu3, w4) + b4 + epsilon
  train_prediction = tf.nn.softmax(train_logit)
    
  valid_relu1= tf.nn.relu(tf.matmul(valid, w1) + b1)
  valid_relu2= tf.nn.relu(tf.matmul(valid_relu1, w2) + b2)
  valid_relu3= tf.nn.relu(tf.matmul(valid_relu2, w3) + b3)
  valid_logit=  tf.matmul(valid_relu3, w4) + b4 + epsilon
  valid_prediction = tf.nn.softmax(valid_logit)

  test_relu1= tf.nn.relu(tf.matmul(test, w1) + b1)
  test_relu2= tf.nn.relu(tf.matmul(test_relu1, w2) + b2)
  test_relu3= tf.nn.relu(tf.matmul(test_relu2, w3) + b3)
  test_logit= tf.matmul(test_relu3, w4) + b4 + epsilon
  test_prediction = tf.nn.softmax(test_logit)
print("graph ready")

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  for step in range(num_steps):
    offset = (step * batch_size) % (train_size - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

graph ready
Minibatch loss at step 0: 37933.648438
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 100: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 200: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%


KeyboardInterrupt: 